In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [5]:
customers = pd.read_csv(r"C:\Users\ADMIN\Downloads\Customers.csv")
products = pd.read_csv(r"C:\Users\ADMIN\Downloads\Products.csv")
transactions = pd.read_csv(r"C:\Users\ADMIN\Downloads\Transactions.csv")


combined_data = pd.merge(transactions, products, on="ProductID", how="left")
combined_data = pd.merge(combined_data, customers, on="CustomerID", how="left")


combined_data.head()


,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price_x,ProductName,Category,Price_y,CustomerName,Region,SignupDate
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,300.68,ComfortLiving Bluetooth Speaker,Electronics,300.68,Andrea Jenkins,Europe,2022-12-03
1,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,300.68,ComfortLiving Bluetooth Speaker,Electronics,300.68,Brittany Harvey,Asia,2024-09-04
2,T00166,C0127,P067,2024-04-25 07:38:55,1,300.68,300.68,ComfortLiving Bluetooth Speaker,Electronics,300.68,Kathryn Stevens,Europe,2024-04-04
3,T00272,C0087,P067,2024-03-26 22:55:37,2,601.36,300.68,ComfortLiving Bluetooth Speaker,Electronics,300.68,Travis Campbell,South America,2024-04-11
4,T00363,C0070,P067,2024-03-21 15:10:10,3,902.04,300.68,ComfortLiving Bluetooth Speaker,Electronics,300.68,Timothy Perez,Europe,2022-03-15


In [9]:
pivot_table = combined_data.pivot_table(index="CustomerID", columns="ProductID", values="Quantity", fill_value=0)
pivot_table.head()

ProductID,P001,P002,P003,P004,P005,P006,P007,P008,P009,P010,...,P091,P092,P093,P094,P095,P096,P097,P098,P099,P100
CustomerID,,,,,,,,,,,,,,,,,,,,,
C0001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
C0002,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
C0003,0.0,4.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C0004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0
C0005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = cosine_similarity(pivot_table)

similarity_df = pd.DataFrame(similarity_matrix, index=pivot_table.index, columns=pivot_table.index)

similarity_df.head()


CustomerID,C0001,C0002,C0003,C0004,C0005,C0006,C0007,C0008,C0009,C0010,...,C0191,C0192,C0193,C0194,C0195,C0196,C0197,C0198,C0199,C0200
CustomerID,,,,,,,,,,,,,,,,,,,,,
C0001,1.0,0.0,0.000000,0.000000,0.000000,0.0,0.214834,0.000000,0.0,0.000000,...,0.061721,0.000000,0.0,0.469668,0.0,0.0,0.0,0.0,0.438178,0.0
C0002,0.0,1.0,0.000000,0.000000,0.000000,0.0,0.000000,0.262071,0.0,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
C0003,0.0,0.0,1.000000,0.097980,0.308697,0.0,0.000000,0.313786,0.0,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
C0004,0.0,0.0,0.097980,1.000000,0.168034,0.0,0.000000,0.048038,0.0,0.149854,...,0.000000,0.000000,0.0,0.079212,0.0,0.0,0.0,0.0,0.092376,0.0
C0005,0.0,0.0,0.308697,0.168034,1.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.325396,0.0,0.083189,0.0,0.0,0.0,0.0,0.000000,0.0


In [15]:
def get_top_3_similar(customers_df, customer_id):
    similar_customers = customers_df[customer_id].sort_values(ascending=False)[1:4] 
    return [(idx, score) for idx, score in similar_customers.items()]


In [17]:
lookalike_dict = {}

# top 3 similar customers for the first 20 customers.
for customer_id in pivot_table.index[:20]:
    lookalike_dict[customer_id] = get_top_3_similar(similarity_df, customer_id)

for key, value in lookalike_dict.items():
    print(f"Customer: {key}, Similar Customers: {value}")


Customer: C0001, Similar Customers: [('C0097', 0.5477225575051661), ('C0194', 0.469668218313862), ('C0199', 0.4381780460041329)]
Customer: C0002, Similar Customers: [('C0091', 0.3801987652174059), ('C0030', 0.37282185960072), ('C0071', 0.329914439536929)]
Customer: C0003, Similar Customers: [('C0134', 0.5199469468957452), ('C0181', 0.5175973113765044), ('C0144', 0.39999999999999997)]
Customer: C0004, Similar Customers: [('C0070', 0.4988876515698588), ('C0132', 0.3843075691322091), ('C0063', 0.3360672201667223)]
Customer: C0005, Similar Customers: [('C0096', 0.6482037235521645), ('C0055', 0.5144957554275265), ('C0064', 0.3328770246548891)]
Customer: C0006, Similar Customers: [('C0058', 0.6488856845230502), ('C0040', 0.5803810000880093), ('C0196', 0.4643048000704075)]
Customer: C0007, Similar Customers: [('C0020', 0.5883484054145521), ('C0079', 0.49613893835683387), ('C0026', 0.36313651960128146)]
Customer: C0008, Similar Customers: [('C0144', 0.39223227027636803), ('C0088', 0.3396831102

In [19]:
lookalike_data = {
    "CustomerID": list(lookalike_dict.keys()),
    "SimilarCustomers": [str(v) for v in lookalike_dict.values()],
}

lookalike_df = pd.DataFrame(lookalike_data)
lookalike_df.to_csv("Tejas_K_Lookalike.csv", index=False)

print("Lookalike.csv saved successfully!")


Lookalike.csv saved successfully!


In [27]:
lookalike_data = pd.read_csv("Tejas_K_Lookalike.csv")

lookalike_data.head()

,CustomerID,SimilarCustomers
0,C0001,"[('C0097', 0.5477225575051661), ('C0194', 0.46..."
1,C0002,"[('C0091', 0.3801987652174059), ('C0030', 0.37..."
2,C0003,"[('C0134', 0.5199469468957452), ('C0181', 0.51..."
3,C0004,"[('C0070', 0.4988876515698588), ('C0132', 0.38..."
4,C0005,"[('C0096', 0.6482037235521645), ('C0055', 0.51..."
